# Fast Binary Logistic Regression (FBLR)

In [ ]:
from platform import python_version

In [ ]:
print('Python version : ', python_version())

# Logistic Regression using Scikit-learn

In [ ]:
applyScaling = False

scatterMatrix = False

showRAMinfoScikitlearn = True
showRAMinfoProposedmethod = True

maxTrainRowCount = None
maxTestRowCount = None

In [ ]:
import numpy as np

In [ ]:
fileName = 'diabetes'
#fileName = 'hepmass'

In [ ]:
import psutil

In [ ]:
ram_info_data_start = psutil.virtual_memory()

X_train = np.load(fileName + '_X_train.npy')
X_test = np.load(fileName + '_X_test.npy')

y_train = np.load(fileName + '_y_train.npy')
y_test = np.load(fileName + '_y_test.npy')

ram_info_data_end = psutil.virtual_memory()

usedMemoryForData = (ram_info_data_end.used - ram_info_data_start.used) / 1024 / 1024 / 1024

print(f"Used memory (for data): {usedMemoryForData:.8f} GB") 

In [ ]:
if (maxTrainRowCount is not None):
    trainSize = min(X_train.shape[0], maxTrainRowCount)
    
    X_train = X_train[:trainSize, :]
    y_train = y_train[:trainSize]

if (maxTestRowCount is not None):
    testSize = min(X_test.shape[0],  maxTestRowCount)
    
    X_test = X_test[:testSize, :]
    y_test = y_test[:testSize]    

In [ ]:
print('X_train : ', X_train.shape)
print('y_train : ', y_train.shape)
print()

print('X_test : ', X_test.shape)
print('y_test : ', y_test.shape)

In [ ]:
print('Train 0-class ', np.sum(y_train == 0))
print('Train 1-class ', np.sum(y_train == 1))
print()

print('Test 0-class ', np.sum(y_test == 0))
print('Test 1-class ', np.sum(y_test == 1))

In [ ]:
import math
from sklearn.preprocessing import RobustScaler

if (applyScaling):
    scaler = RobustScaler()
    scaler.fit(X_train)

    X_train = scaler.transform(X_train) * (2.0 * math.pi)
    X_test = scaler.transform(X_test) * (2.0 * math.pi)
    
    X_mu = np.mean(X_train, axis=0).reshape(1, -1)
    X_train -= X_mu
    X_test -= X_mu     

In [ ]:
print('X_train : ', X_train.shape)
print('y_train : ', y_train.shape)
print()

print('X_test : ', X_test.shape)
print('y_test : ', y_test.shape)

In [ ]:
n = X_train.shape[0]
d = X_train.shape[1]

print('Train data')
print('----------')
print('n = ', n)
print('d = ', d)

In [ ]:
maximumExperimentCount = 1000
experimentCount = round((100000 * 100) / (n * d) + 0.5)

print('Experiment count = ', experimentCount)

In [ ]:
randomSeedValue = 12345

np.random.seed(randomSeedValue)

In [ ]:
import time

In [ ]:
start_time_all = time.time()

In [ ]:
# import the class
from sklearn.linear_model import LogisticRegression

ScikitlearnMaximumIteration = 250

start_time = time.time()

#solver="lbfgs"             # .x   (default solver, memory efficient)
solver="liblinear"          # .x
#solver="newton-cg"         # .x
#solver="newton-cholesky"   # .x 
#solver="sag"               # .x
#solver="saga"              # .x

ram_info_start = None
ram_info_end = None 

for expNo in range(experimentCount):
    if (showRAMinfoScikitlearn):
        ram_info_start = psutil.virtual_memory()
    
    # instantiate the model (using the default parameters)
    logreg = LogisticRegression(max_iter=ScikitlearnMaximumIteration, fit_intercept=True, solver=solver)
    
    # fit the model with data
    logreg.fit(X_train, np.ravel(y_train, order='C'))
    
    if (showRAMinfoScikitlearn):
        ram_info_end = psutil.virtual_memory()
        showRAMinfoScikitlearn = False
    
scikitlearn_elapsed_time = (time.time() - start_time) / experimentCount

print("Execution time = %.6f seconds" % scikitlearn_elapsed_time)

In [ ]:
usedMemoryScikitlearn = None
if (ram_info_start is not None and ram_info_end is not None):
    print()
    usedMemoryScikitlearn = (ram_info_end.used - ram_info_start.used) / 1024 / 1024 / 1024
    usedMemoryScikitlearnWithInputData = usedMemoryForData + usedMemoryScikitlearn
    
    print(f"Used memory: {usedMemoryScikitlearnWithInputData:.8f} GB")

In [ ]:
lr_b = logreg.intercept_
lr_w = logreg.coef_[0][:, np.newaxis]

print('Scikit-learn Weights:')
print(lr_b, np.round(lr_w.flatten(), 5))

In [ ]:
y_proba = logreg.predict_proba(X_test)

print(np.min(y_proba[:,0]), np.max(y_proba[:,0]))
print()

print(y_proba[:,0])

In [ ]:
y_pred = logreg.predict(X_test)

print(y_pred)

In [ ]:
y_proba = logreg.predict_proba(X_test)
y_pred = np.multiply(y_proba[:,1] >= 0.5, 1)

print(y_pred)

In [ ]:
logreg = None

In [ ]:
from sklearn import metrics

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

scikit_accuracy = metrics.accuracy_score(y_test, y_pred)
scikit_precision = metrics.precision_score(y_test, y_pred)
scikit_recall = metrics.recall_score(y_test, y_pred)
scikit_f1 = metrics.f1_score(y_test, y_pred)

print("Accuracy  : %.4f" % scikit_accuracy)
print("Precision : %.4f" % scikit_precision)
print("Recall    : %.4f" % scikit_recall)
print("F1-Score  : %.4f" % scikit_f1)

scikit_fpr, scikit_tpr, _ = metrics.roc_curve(y_test,  y_proba[:,1])
scikit_auc = round(metrics.roc_auc_score(y_test, y_proba[:,1]), 4)
print("ROC AUC   : %.4f" % scikit_auc)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(scikit_fpr, scikit_tpr, label="Scikit AUC=" + str(scikit_auc), color='g')
plt.title('Logistic Regression using Scikit-learn')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.grid()
plt.show()

# Linear Regression

In [ ]:
ram_info_start = psutil.virtual_memory()

X_train_biased = np.hstack((np.ones((X_train.shape[0], 1)), X_train))

gamma = 1e-8
d = X_train_biased.shape[1]
A = (X_train_biased.T).dot(X_train_biased) + gamma*np.identity(d)
b = (X_train_biased.T).dot(y_train)
lreg_w = np.linalg.solve(A, b)

ram_info_end = psutil.virtual_memory()

usedMemoryLinearRegression = (ram_info_end.used - ram_info_start.used) / 1024 / 1024 / 1024
usedMemoryLinearRegressionWithInputData = usedMemoryForData + usedMemoryLinearRegression

print(f"Used memory: {usedMemoryLinearRegressionWithInputData:.8f} GB")

In [ ]:
print(lreg_w)

In [ ]:
X_test_biased = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

y_hat = X_test_biased.dot(lreg_w)
y_pred = np.multiply(y_hat > 0, 1).flatten()

print(y_pred)

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

print("            Linear Regression      Scikit-learn")
print("            -----------------      ------------")
print("Accuracy  :    %.4f          /      %.4f " % (metrics.accuracy_score(y_test, y_pred), scikit_accuracy))
print("Precision :    %.4f          /      %.4f " % (metrics.precision_score(y_test, y_pred), scikit_precision))
print("Recall    :    %.4f          /      %.4f " % (metrics.recall_score(y_test, y_pred), scikit_recall))
print("F1-Score  :    %.4f          /      %.4f " % (metrics.f1_score(y_test, y_pred), scikit_f1))

# Logistic Regression using Proposed Model

In [ ]:
import copy

In [ ]:
import scipy as sp
from scipy.linalg import cho_factor, cho_solve

In [ ]:
epsilon = 1e-10
lambda_ssr = 0  #0.1
f = 0.0
gamma = 0  #1.0
convergenceTolerance = 1e-3

In [ ]:
def fastLogisticRegression(X_train, y_train, fit_intercept = True, epsilon = 1e-10, lambda_ssr = 0, f = 0, gamma = 0, convergenceTolerance = 1e-3, minimumIteration = 2, maximumIteration = 10, verbose=False):
        
    if (verbose):
        costList = []
        
    if (fit_intercept):
        X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
        
    n = X_train.shape[0]
    d = X_train.shape[1]
        
    I = np.identity(d)
    
    beta = 1e-8
    A = (X_train.T).dot(X_train) + beta * I
    b = (X_train.T).dot(y_train)
    w_0 = np.linalg.solve(A, b)    
    
    doRegularization = (lambda_ssr > 0 or gamma > 0)

    log2 = math.log(2.0)
    t = log2
    q = 0.5

    v = (1/n)*(X_train.T).dot(y_train - q)

    w = copy.deepcopy(w_0)
    for iteration in range(maximumIteration+1):
        w_hat = copy.deepcopy(w)

        o = X_train.dot(w_hat)
        z = (np.log(1.0 + np.exp(o)) - log2 - 0.5*o) / (o*o + epsilon)

        # calculate cost
        if (verbose):
            cost = (1/n)*np.sum(o*(z*o + (q - y_train))) + t
            costList.append(cost.item())

        # weight update
        if (doRegularization):
            p = np.ones(d)
            if (interceptColumnIndex >= 0 and interceptColumnIndex < d):
                p[interceptColumnIndex] = 0

            h = p / (np.abs(w_hat)**(2-f) + epsilon)
            H = np.diag(h.flatten())

            # A = (2/n) * (X_train.T Z X_train) + (lambda_ssr/d)*I + (gamma/d)*H
            A = (2/n) * np.multiply(z, X_train.T).dot(X_train) + (lambda_ssr/d)*I + (gamma/d)*H
            b = (lambda_ssr/d) * w + v
            w = np.linalg.solve(A, b)
        else:
            # A = (2/n) * (X_train.T Z X_train)
            A = (2/n) * np.multiply(z, X_train.T).dot(X_train)
            w = np.linalg.solve(A, v)
            
        change = np.max(np.abs(w - w_hat))
        if (iteration >= minimumIteration and change <= convergenceTolerance):
            break        
       
    if (verbose):
        return w, w_0, costList
    else:
        return w        

In [ ]:
if (showRAMinfoProposedmethod):
    ram_info_start = psutil.virtual_memory()

    w = fastLogisticRegression(X_train, y_train, verbose=False)
    
    ram_info_end = psutil.virtual_memory()
    
    usedMemoryProposedMethod = (ram_info_end.used - ram_info_start.used) / 1024 / 1024 / 1024
    usedMemoryProposedMethodWithInputData = usedMemoryForData + usedMemoryProposedMethod
    
    if (usedMemoryScikitlearn is not None):
        print(f"Proposed Method / Scikit-learn (memory usage): {(usedMemoryProposedMethodWithInputData / usedMemoryScikitlearnWithInputData):.3f}x")
        print()
        
        print(f"Used memory (Scikit-learn): {usedMemoryScikitlearnWithInputData:.8f} GB") 

    print(f"Used memory (Proposed Method): {usedMemoryProposedMethodWithInputData:.8f} GB")

In [ ]:
start_time = time.time()

for expNo in range(experimentCount):
    verbose = (expNo == 0)
    if (verbose):
        w, w_0, costList = fastLogisticRegression(X_train, y_train, verbose=True)
    else:
        w = fastLogisticRegression(X_train, y_train, verbose=False)
    
proposedMethodElapsedTime = ( time.time() - start_time) / experimentCount

speedup = round(scikitlearn_elapsed_time / proposedMethodElapsedTime, 2)

print('Converged in %s iterations ' % len(costList))
print("Scikit-learn execution time = %.6f seconds" % scikitlearn_elapsed_time)
print('Proposed Method execution time = %.6f seconds' % proposedMethodElapsedTime)
print('Obtained speedup is %5.2fx' % speedup)

In [ ]:
np.set_printoptions(precision=5)

In [ ]:
print('Scikit-learn Weights:')
print(lr_b, end=' ')
for data in lr_w.flatten():
    print('{:14.9f}'.format(data), end="")
print()
print()

print('Least Square (Initial) Weights:')
for data in w_0.flatten():
    print('{:14.9f}'.format(data), end="")    
print()
print()

print('Proposed Method Weights:')
for data in w.flatten():
    print('{:14.9f}'.format(data), end="")

In [ ]:
print('Scikit-learn Weights (normalized):')
for data in (lr_w / np.linalg.norm(lr_w)).flatten():
    print('{:14.9f}'.format(data), end="")

In [ ]:
w_normalized = w / (np.linalg.norm(w) + np.min(np.abs(w)))
print('Proposed Method Weights (normalized):')
for data in (w_normalized).flatten():
    print('{:14.9f}'.format(data), end="")

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(costList)
plt.title('Cost graph for the Proposed Method')
plt.ylabel('Cost')
plt.xlabel('Iteration')
plt.grid()
plt.show()

In [ ]:
X_test_biased = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

e = np.exp(X_test_biased.dot(w))
y_proba = (e / (1.0 + e)).flatten()
y_pred = np.multiply(y_proba >= 0.5, 1.0)

In [ ]:
print(np.min(y_proba), np.max(y_proba))
print()

print(y_proba)

In [ ]:
print(y_pred)

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

print("            Proposed      Scikit-learn")
print("            --------      ------------")
print("Accuracy  :  %.4f    /    %.4f " % (metrics.accuracy_score(y_test, y_pred), scikit_accuracy))
print("Precision :  %.4f    /    %.4f " % (metrics.precision_score(y_test, y_pred), scikit_precision))
print("Recall    :  %.4f    /    %.4f " % (metrics.recall_score(y_test, y_pred), scikit_recall))
print("F1-Score  :  %.4f    /    %.4f " % (metrics.f1_score(y_test, y_pred), scikit_f1))

fpr, tpr, _ = metrics.roc_curve(y_test,  y_proba)
auc = round(metrics.roc_auc_score(y_test, y_proba), 4)
print("ROC AUC   :  %.4f    /    %.4f " % (auc, scikit_auc))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(scikit_fpr, scikit_tpr, label="Scikit AUC=" + str(scikit_auc), color='g')
plt.plot(fpr, tpr, label="LLRR AUC=" + str(auc), color='r')
plt.title('Fast Large-scale Logistic Regression (FLLR), Speedup is: ' + str(speedup) + 'x')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.grid()
plt.show()

# Logistic Regression using Proposed Model with Low-Rank Approximation

In [ ]:
from sklearn.utils.extmath import randomized_svd

In [ ]:
from customClassifier import FastLogisticRegressionLowRank

In [ ]:
if (showRAMinfoProposedmethod):
    ram_info_start = psutil.virtual_memory()

    fblr = FastLogisticRegressionLowRank()
    fblr.fit(X_train, y_train)

    ram_info_end = psutil.virtual_memory()
    usedMemoryProposedMethodLowRank = (ram_info_end.used - ram_info_start.used) / 1024 / 1024 / 1024
    usedMemoryProposedMethodLowRankWithInputData = usedMemoryForData + usedMemoryProposedMethodLowRank        
        
    if (usedMemoryScikitlearn is not None):
        print(f"Proposed Method / Scikit-learn (memory usage): {(usedMemoryProposedMethodWithInputData / usedMemoryScikitlearnWithInputData):.3f}x")
        print()

        print(f"Used memory (Scikit-learn): {usedMemoryScikitlearnWithInputData:.8f} GB") 

    print(f"Used memory (Proposed Method): {usedMemoryProposedMethodLowRankWithInputData:.8f} GB")

In [ ]:
start_time = time.time()

for expNo in range(experimentCount):
    fblr = FastLogisticRegressionLowRank()
    fblr.fit(X_train, y_train)
    
    fblr_b = fblr.intercept_
    fblr_w = fblr.coef_
    
proposedMethodElapsedTimeLowRank = ( time.time() - start_time) / experimentCount

speedup = round(scikitlearn_elapsed_time / proposedMethodElapsedTimeLowRank, 2)

print("Scikit-learn execution time = %.6f seconds" % scikitlearn_elapsed_time)
print('Proposed Method Low-Rank execution time = %.6f seconds' % proposedMethodElapsedTimeLowRank)
print('Obtained speedup is %5.2fx' % speedup)

In [ ]:
print("Data dimension = %d " % d)
#print("Low-Rank (r) = %d " % fblr.r)

In [ ]:
np.set_printoptions(precision=5)

In [ ]:
print('Scikit-learn Weights:')
print(lr_b, end=' ')
for data in lr_w.flatten():
    print('{:14.9f}'.format(data), end="")
print()
print()

print('Least Square (Initial) Weights (Low-Rank):')
for data in w_0.flatten():
    print('{:14.9f}'.format(data), end="")    
print()
print()

print('Proposed Method Weights (Low-Rank):')
print(fblr_b, end=' ')
for data in fblr_w.flatten():
    print('{:14.9f}'.format(data), end="")

In [ ]:
print('Scikit-learn Weights (normalized):')
for data in (lr_w / np.linalg.norm(lr_w)).flatten():
    print('{:14.9f}'.format(data), end="")

In [ ]:
w_normalized = w / (np.linalg.norm(w) + np.min(np.abs(w)))
print('Proposed Method Weights (normalized):')
for data in (w_normalized).flatten():
    print('{:14.9f}'.format(data), end="")

In [ ]:
y_proba = fblr.predict_proba(X_test)
y_pred = fblr.predict(X_test)

In [ ]:
print(np.min(y_proba), np.max(y_proba))
print()

print(y_proba)

In [ ]:
print(y_pred)

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

print("            Proposed      Scikit-learn")
print("            --------      ------------")
print("Accuracy  :  %.4f    /    %.4f " % (metrics.accuracy_score(y_test, y_pred), scikit_accuracy))
print("Precision :  %.4f    /    %.4f " % (metrics.precision_score(y_test, y_pred), scikit_precision))
print("Recall    :  %.4f    /    %.4f " % (metrics.recall_score(y_test, y_pred), scikit_recall))
print("F1-Score  :  %.4f    /    %.4f " % (metrics.f1_score(y_test, y_pred), scikit_f1))

fpr, tpr, _ = metrics.roc_curve(y_test,  y_proba[:,1])
auc = round(metrics.roc_auc_score(y_test, y_proba[:,1]), 4)
print("ROC AUC   :  %.4f    /    %.4f " % (auc, scikit_auc))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(scikit_fpr, scikit_tpr, label="Scikit AUC=" + str(scikit_auc), color='g')
plt.plot(fpr, tpr, label="FLLR AUC=" + str(auc), color='r')
plt.title('Fast Large-scale Logistic Regression (FLLR) - Low-Rank, Speedup is: ' + str(speedup) + 'x')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.grid()
plt.show()

In [ ]:
scikitlearn_elapsed_time_all = (time.time() - start_time_all)

print("Execution time of all the experiments = %.6f seconds" % scikitlearn_elapsed_time_all)

# Regularization Paths

In [ ]:
import sys

In [ ]:
gammaCount = 25

In [ ]:
# Scikit-learn (paths)
start_time = time.time()

wList = []
gammaList = []

best_C = None
maximum_accuracy = 0.0

maxGamma = 3.0

for gamma in np.linspace(1e-4, maxGamma*d, gammaCount):
    sys.stdout.write('.')
    
    # instantiate the model (using the default parameters)
    # intercept_scaling = 10
    logreg = LogisticRegression(max_iter=ScikitlearnMaximumIteration, fit_intercept=True, solver=solver, penalty='l1', intercept_scaling = 10, C=1/gamma)
    logreg.fit(X_train, np.ravel(y_train, order='C'))    
    y_pred = logreg.predict(X_test) 
    
    accuracy = metrics.accuracy_score(y_test, y_pred)
    if (accuracy > maximum_accuracy):
        maximum_accuracy = accuracy
        best_C = 1 / gamma
    
    lr_w = np.hstack((logreg.intercept_, logreg.coef_[0].flatten()))
    
    wList.append(lr_w)
    gammaList.append(gamma)

ScikitLearnLogisticRegresssionElapsedTimeCV = ( time.time() - start_time)

print()
print()

print('Scikit-learn Logistic Regresssion CV execution time = %.6f seconds' % ScikitLearnLogisticRegresssionElapsedTimeCV)

In [ ]:
print('Best C = ', best_C)
print('Best gamma = ', 1 / best_C)
print('Maximum accuracy = ', maximum_accuracy)

In [ ]:
legendList = []
d = len(wList[0])
for k in range(d):
    if (k == 0):
        legendList.append('w' + str(k) + ' (intercept)')
    else:
        legendList.append('w' + str(k))

In [ ]:
# Show regularization paths
plt.figure(1, figsize=(11, 9)) 
plt.plot(gammaList, wList)
plt.xlabel('C')
plt.ylabel('weights')
plt.title('Coefficients with respect to Gamma (Scikit-Learn)')
plt.legend(legendList, loc="lower right") 
plt.grid()
plt.show()

In [ ]:
# Proposed method (paths)
start_time = time.time()

wList = []
gammaList = []

best_gamma = None
maximum_accuracy = 0.0

for gamma in np.linspace(1e-8, 0.5*maxGamma, gammaCount):
    sys.stdout.write('.')
    
    fblr = FastLogisticRegressionLowRank(lambda_ssr = 1.0, f=1.0, gamma=gamma)
    fblr.fit(X_train, y_train)
    
    lr_w = np.hstack((fblr.intercept_, fblr.coef_))

    y_pred = fblr.predict(X_test)
    
    accuracy = metrics.accuracy_score(y_test, y_pred)
    if (accuracy > maximum_accuracy):
        maximum_accuracy = accuracy
        best_gamma = gamma
        
    wList.append(lr_w)
    gammaList.append(gamma)

proposedMethodElapsedTimeLowRankCV = ( time.time() - start_time)

print()
print()

print('Proposed Method Low-Rank CV execution time = %.6f seconds' % proposedMethodElapsedTimeLowRankCV)

In [ ]:
print('Best gamma = ', best_gamma)
print('Maximum accuracy = ', maximum_accuracy)

In [ ]:
legendList = []
d = len(wList[0])
for k in range(d):
    if (k == 0):
        legendList.append('w' + str(k) + ' (intercept)')
    else:
        legendList.append('w' + str(k))

In [ ]:
# Show regularization paths
plt.figure(1, figsize=(11, 6)) 
plt.plot(gammaList, wList)
plt.xlabel('gamma')
plt.ylabel('weights')
plt.title('Coefficients with respect to Gamma (Proposed Method)')
plt.legend(legendList, loc="lower right") 
plt.grid()
plt.show()

In [ ]:
speedup = ScikitLearnLogisticRegresssionElapsedTimeCV / proposedMethodElapsedTimeLowRankCV

print('Obtained speedup is %fX' % speedup)